In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import transforms
import data
from model.unet import U_Net
import matplotlib.pyplot as plt
import torch
from torchsummary import summary
import glob
from sklearn.model_selection import train_test_split

/Users/maochenhui/miniforge3/envs/DL/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
model = U_Net()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
        conv_block-7         [-1, 64, 224, 224]               0
         MaxPool2d-8         [-1, 64, 112, 112]               0
            Conv2d-9        [-1, 128, 112, 112]          73,856
      BatchNorm2d-10        [-1, 128, 112, 112]             256
             ReLU-11        [-1, 128, 112, 112]               0
           Conv2d-12        [-1, 128, 112, 112]         147,584
      BatchNorm2d-13        [-1, 128, 112, 112]             256
             ReLU-14        [-1, 128, 1

In [3]:
SCANS_DIR = "input/scans/scans/"
TRUTH_DIR = "input/ground-truth-pixel/ground-truth-pixel/"
scan_files = glob.glob(SCANS_DIR+'*.png')
scan_files = sorted(scan_files)
scan_files = list(filter(lambda x:"(" not in x, scan_files))
scan_files_dev = scan_files[:400]

scan_files_test = scan_files[400:]
truth_files = glob.glob(TRUTH_DIR+'*.png')
truth_files = sorted(truth_files)
truth_files = list(filter(lambda x:"(" not in x, truth_files))

scan_files_train, scan_files_val, truth_files_train, truth_files_val = train_test_split(scan_files_dev, truth_files, test_size=0.3)

In [4]:
from torch.utils.data import DataLoader
train_data = data.CustomDataset(scans = scan_files_train, ground_truth=truth_files_train, transforms=transforms.default_transform["train"])
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

val_data = data.CustomDataset(scans=scan_files_val, ground_truth=truth_files_val, transforms=transforms.default_transform["val"])
val_loader = DataLoader(val_data, batch_size=32)

In [5]:
from train import Trainer
train = (scan_files_train, truth_files_train)
val = (scan_files_val, truth_files_val)
trainer = Trainer(train, val, model)

In [ ]:
trainer()

Start Training
At epoch: 0


 11%|████▉                                       | 1/9 [01:43<13:44, 103.05s/it]

train loss: 3.485365629196167
Validation loss decreased (inf --> 3.485366).  Saving model ...


 22%|█████████▊                                  | 2/9 [03:28<12:10, 104.31s/it]

train loss: 3.2616878747940063
Validation loss decreased (3.485366 --> 3.261688).  Saving model ...
